In [20]:
from z3 import *

from itertools import combinations
from z3 import *
from utils import *
import math
import time

In [21]:
#z3.set_param('sat.random_seed', 42)

In [22]:
file_name='inst13.dat'
file = open('./Instances/'+file_name, 'r')
splitted_file = file.read().split('\n')
m = int(splitted_file[0])
n = int(splitted_file[1])
cpt = list(map(int, splitted_file[2].split(' ')))
sz = list(map(int, splitted_file[3].split(' ')))
D = [list(map(int, line.strip().split(' '))) for line in splitted_file[4:(n+5)]]
print(n)

47


In [23]:
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one_np(bool_vars, name = ""):
    return And(at_least_one_np(bool_vars), at_most_one_np(bool_vars, name))

In [24]:
def at_least_one_seq(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_seq(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

In [25]:
def at_least_k_seq(bool_vars, k, name):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k_seq(bool_vars, k, name):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))
        for j in range(1, k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))   
    return And(constraints)

def exactly_k_seq(bool_vars, k, name):
    return And(at_most_k_seq(bool_vars, k, name+'1'), at_least_k_seq(bool_vars, k, name))

In [26]:
def print_items_for_each_courier(couriers_items, items_predecessors, sz, cpt):
    # Creare un dizionario per mappare ogni corriere ai suoi pacchi
    couriers_dict = {}
    for i, j in couriers_items:
        if i not in couriers_dict:
            couriers_dict[i] = []
        couriers_dict[i].append(j)
    #print('cour_dict', couriers_dict)

    n=0
    for value in couriers_dict.values():
        n = n+len(value)
    #print(n)


    # Creare un altro dizionario per mappare ogni pacco al suo predecessore
    
    predecessors_dict = {i: j for j, i in items_predecessors}
    #print('pred_dict', predecessors_dict)
    
    # Per ogni corriere, seguire la catena di predecessori per ottenere l'ordine corretto dei pacchi
    for courier, items in couriers_dict.items():
        print(f"Corriere {courier}: ", end='')
        item = n + courier
        order = [item]
        while item in predecessors_dict and predecessors_dict[item] not in order:
            item = predecessors_dict[item]
            order.append(item)
        item = predecessors_dict[item]
        order.append(item)
        print(" -> ".join(map(str, order)), end='   ')
        print('Peso trasportato: ', end='')
        weight=0
        for i in couriers_dict[courier]:
            weight += sz[i]

        print(weight, '/', cpt[courier])

In [27]:
def max_z3(vec):
  m = vec[0]
  for value in vec[1:]:
    m = If(value > m, value, m)
  return m

In [28]:
'''n packs
   m couriers
   D matix int(n+1,m+1)
   sz array of item's sizes (len = n)
   cpt array of couriers' capacities (len = m)
'''
def mcp(n, m, D, sz, cpt):

   #create Solver
   solv= Solver()

   '''
      vehicle matrix 
      col = packs
      rows = courriers 
      each rows represent the list of packs foreach courrier
   ''' 
   #create a [m x (n+m)] matrix v. v[i][j]==True if the i-th courier takes the j-th item (or starting/ending point)
   v=[[Bool(f"vehicle({i})_{j}")for j in range(n+m)]for i in range(m)]

   #each pack have to be carried by exactly one courrier
   for j in range(n):
      solv.add(exactly_one_seq([v[i][j] for i in range(m)], f"v_{j}"))

   for j in range(m):
      for i in range(m):
         if j==i:
            solv.add(v[i][n+j])
         else:
            solv.add(Not(v[i][n+j]))
   
   
   #Constraint peso
   for c in range(m):
      solv.add(Sum([If(v[c][i], 1, 0)*sz[i] for i in range(n)]) <= cpt[c])
   
   '''
      pred matrix  
   '''
   #pred[i][j] = true if the j-th item is the predecessor of the i-th item. 
   #The n+i column in the matrix is the starting point of the i-th courier. The n+i row in the matrix is the ending point of the i-th courier
   pred = [[Bool(f"pred({i})_{j}")for j in range(n+m)]for i in range(n+m)]

   #Each item/ending point has exactly one predecessor and each item/starting point is predecessor of exactly one other item. 
   #i=0 to i=n+m-1
   #j=0 to j=n+m-1 
   for i in range(n+m):
      col_i = []
      for j in range(n+m):
         col_i += [pred[j][i]]

      solv.add(exactly_one_seq(col_i, f"PC_{i}"))
      solv.add(exactly_one_seq(pred[i], f"PR_{i}"))

   #Constraint coerenza courier - predecessore/successore
   for c in range(m):
      solv.add(And(  [Implies(And([ v[c][i], pred[i][j]] ) , v[c][j]) for i in range(n+m) for j in range(n+m)]  ))

   #FUNZIONA. WOW :D
   #solv.add(And( [Implies( And([pred[i][n], pred[j][n]]) , And([ Not(And(v[k][i], v[k][j])) for k in range(m)])) for i in range(n) for j in range(n) if i<j])) #Se due items hanno come predecessore il deposito allora devono essere presi da due veicoli diversi

   
   '''
      no_loops 
   '''
   no_loops = IntVector('no_loops', n)
   solv.add(And([Implies(pred[i][j], no_loops[i]>no_loops[j]) for i in range(n) for j in range(n)]))
   #solv.add(no_loops[0]==1)

   #Distance
   dist_vector = IntVector('dist_vector', m)
   for courier in range(m):
      starting_point = [If(pred[item][n+courier], 1, 0)*D[n][item] for item in range(n)]
      ending_point = [If(pred[n+courier][item], 1, 0)*D[item][n] for item in range(n)]
      mid_points = [If(And(v[courier][i], pred[i][j]), 1, 0)*D[j][i] for i in range(n) for j in range(n)]
      distance_of_this_path = Sum(starting_point + mid_points + ending_point)
      solv.add(dist_vector[courier] == distance_of_this_path)
   max_dist = Int('max_dist')
   #max_dist = max_z3(dist_vector)
   solv.add(max_dist == max_z3(dist_vector))
   
   return solv, max_dist, pred, v


In [29]:
def mcp_small(n, D):

   #create Solver
   solv= Solver()
   
   '''
      pred matrix  
   '''
   #pred[i][j] = true if the j-th item is the predecessor of the i-th item. 
   #The n+i column in the matrix is the starting point of the i-th courier. The n+i row in the matrix is the ending point of the i-th courier
   pred = [[Bool(f"pred({i})_{j}")for j in range(n+1)]for i in range(n+1)]

   #Each item/ending point has exactly one predecessor and each item/starting point is predecessor of exactly one other item. 
   #i=0 to i=n+m-1
   #j=0 to j=n+m-1 
   for i in range(n+1):
      col_i = []
      for j in range(n+1):
         col_i += [pred[j][i]]

      solv.add(exactly_one_seq(col_i, f"PC_{i}"))
      solv.add(exactly_one_seq(pred[i], f"PR_{i}"))
   
   '''
      no_loops 
   '''
   no_loops = IntVector('no_loops', n)
   solv.add(And([Implies(pred[i][j], no_loops[i]>no_loops[j]) for i in range(n) for j in range(n)]))
   #solv.add(no_loops[0]==1)

   #Distance
   
   starting_point = [If(pred[item][n], 1, 0)*D[n][item] for item in range(n)]
   ending_point = [If(pred[n][item], 1, 0)*D[item][n] for item in range(n)]
   mid_points = [If(pred[i][j], 1, 0)*D[j][i] for i in range(n) for j in range(n)]
   distance_of_this_path = Sum(starting_point + mid_points + ending_point)
   max_dist = Int('max_dist')
   solv.add(max_dist == distance_of_this_path)
   
   return solv, max_dist, pred


In [30]:
import numpy as np
import copy

def clustering(D):
    list_of_mins=[]
    n=len(D)-1
    Closest=[]
    for item in range(n): 
        tmp = copy.deepcopy(D[item])
        tmp[item]=1000
        Closest.append(np.argmin(tmp[:-1])) 
        list_of_mins.append(np.min(tmp[:-1]))
    #print(list_of_mins)
    clusters=[]
    already_in_cluster=[]
    for item in range(n):
        if item==Closest[Closest[item]] and (item not in already_in_cluster):
            clusters.append([item, Closest[item]])
            already_in_cluster.append(item)
            already_in_cluster.append(Closest[item])
        elif item!=Closest[Closest[item]]:
            clusters.append([item])
    clusters.append([n])
    D_new=[]
    n_new=len(clusters)-1

    for row in range(n_new): 
        D_new.append([]) 
        for column in range(n_new): 
            if row==column:
                D_new[row].append(0)
            else:
                tmp_dist=[]
                for i in clusters[row]:
                    for j in clusters[column]:
                        tmp_dist.append(D[i][j])
                D_new[row].append(np.mean(tmp_dist))
        tmp_dist=[]
        for i in clusters[row]:
            tmp_dist.append(D[i][n])
        D_new[row].append(np.mean(tmp_dist))
    D_new.append([])

    for column in range(n_new):
        tmp_dist=[]
        for i in clusters[column]:
            tmp_dist.append(D[n][i])
        D_new[n_new].append(np.mean(tmp_dist))
    D_new[n_new].append(0)

    return D_new, clusters

In [31]:
D_opt=D
clusters=[ [i] for i in range(n+1) ]
for i in range(2):
    D_opt, new_clusters=clustering(D_opt)
    old_clusters=clusters
    clusters=[]
    for clus in new_clusters:
        clusters.append([])
        for item in clus:
            for old_item in old_clusters[item]:
                clusters[-1].append(old_item)
print(clusters)
print(len(D_opt))

[[0, 13, 27], [1, 29], [2, 5, 26], [3], [4, 7, 28], [6, 36], [8, 34], [9, 44, 10], [11, 23, 46], [12], [14, 24, 22], [15], [16], [17, 25], [18], [19, 32], [20], [21], [30, 31, 43], [33], [35], [37], [38, 42], [39], [40], [41, 45], [47]]
27


In [46]:
clusters_paths=[]
cluster_path_distances=[]

for cluster in clusters:
    n_cluster = len(cluster)
    if len(cluster)>1:
        D_clus=[]
        for i in cluster:
            D_clus.append([])
            for j in cluster:
                D_clus[-1].append(D[i][j])
            D_clus[-1].append(0)
        D_clus.append([0 for k in range(n_cluster+1)])
        #print(D_clus)
        tmp_solv, max_dist, pred =mcp_small(n = len(cluster), D=D_clus)
        while tmp_solv.check()==sat:
            tmp_model=tmp_solv.model()
            best_dist=tmp_model.evaluate(max_dist).as_long()
            tmp_solv.add(max_dist<best_dist)
        cluster_copy=copy.deepcopy(cluster)
        cluster_copy.append(-1)
        clusters_paths.append([(cluster_copy[i],cluster_copy[j]) for i in range(n_cluster+1) for j in range(n_cluster+1) if tmp_model.evaluate(pred[i][j])])
        cluster_path_distances.append(best_dist)

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


In [33]:
#Modify D_opt?

n_new = len(clusters)-1

first_items_for_clusters=[]
last_item_for_clusters=[]

i=0
for clus in clusters:
    if len(clus)>1:
        path=clusters_paths[i]
        for couple in path:
            if couple[0]==-1:
                last_item_for_clusters.append(couple[1])
            elif couple[1]==-1:
                first_items_for_clusters.append(couple[0])
        i+=1
    else:
        last_item_for_clusters.append(clus[0])
        first_items_for_clusters.append(clus[0])

D_new=[]
for i in range(n_new+1):
    D_new.append([])
    for j in range(n_new+1):
        if j==i:
            D_new[-1].append(0)
        else:
            D_new[-1].append( D[first_items_for_clusters[i]][last_item_for_clusters[j]] )

In [34]:
sz_new=[]
for item in range(n_new):
    sz_new.append(sum([sz[i] for i in clusters[item]]))

In [35]:
starting_time = time.time()
timeout = starting_time + 60*5
check_timeout = timeout-time.time()

solv, max_dist, pred, v = mcp(n_new, m, D_new, sz_new, cpt)
print('builded')
checked_solv = solv.check()
print('checked')
to_print=[check_timeout]

if checked_solv==sat:
    best_model = solv.model()
    best_distance = best_model.evaluate(max_dist).as_long()

    to_print.append(('solution 1) obj = ', best_distance, '; Time used: ', time.time()-starting_time))

    #solv.add( Not( And([pred[i][j] for i in range(n) for j in range(n+m) if best_model.evaluate(pred[i][j])] + [v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])])) )
    solv.add( max_dist < best_distance)
    print('qui')
    check_timeout = timeout-time.time()
    solv.set('timeout', int(check_timeout*1000))
    checked_solv = solv.check()
    print('...')

while time.time() < timeout and checked_solv==sat:

    best_model = solv.model()
    best_distance = best_model.evaluate(max_dist).as_long()

    to_print.append(('Solution 1) Obj = ', best_distance, '; Time used: ', time.time()-starting_time))

    solv.add( max_dist < best_distance)
    #solv.add( Not( And([pred[i][j] for i in range(n+m) for j in range(n+m) if best_model.evaluate(pred[i][j])] + [v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])])) )
    
    check_timeout = timeout-time.time() #time left
    solv.set('timeout', int(check_timeout*1000)) #time left in millisec
    
    checked_solv = solv.check()
    
ending_time = time.time()
print('----------------Recap----------------------')
print('Total time: ', to_print[0])
print('Time used: ', ending_time - starting_time)
print('')
for i in range(len(to_print)-1):
    for j in range(4):
        print(to_print[i+1][j], end='')
    print('')

print('') 
print('Best solution:')   
item_pred, cour_item = [(i,j) for j in range(n+m) for i in range(n+m) if best_model.evaluate(pred[i][j])], [(i, j) for j in range(n) for i in range(m) if best_model.evaluate(v[i][j])]
print_items_for_each_courier(cour_item, item_pred, sz, cpt)
print('Minimized max distance: ', sum(cluster_path_distances))

builded
checked
qui
...
----------------Recap----------------------
Total time:  300.0
Time used:  300.00748467445374

solution 1) obj = 1178; Time used: 2.77095890045166
Solution 1) Obj = 940; Time used: 11.13576078414917
Solution 1) Obj = 863; Time used: 14.928725004196167
Solution 1) Obj = 862; Time used: 15.031031608581543
Solution 1) Obj = 824; Time used: 15.0762300491333
Solution 1) Obj = 810; Time used: 15.140470743179321
Solution 1) Obj = 780; Time used: 15.593956470489502
Solution 1) Obj = 737; Time used: 15.995114088058472
Solution 1) Obj = 722; Time used: 20.733587503433228
Solution 1) Obj = 662; Time used: 20.780815362930298
Solution 1) Obj = 629; Time used: 39.12208390235901
Solution 1) Obj = 625; Time used: 39.18740940093994
Solution 1) Obj = 599; Time used: 178.75156927108765

Best solution:


IndexError: list index out of range

In [47]:
sum(cluster_path_distances)

270

In [104]:
starting_time = time.time()
timeout = starting_time + 60*5
check_timeout = timeout-time.time()
print('start')
solv, max_dist, pred, v = mcp(n,m,D,sz,cpt)
print('qui')
checked_solv = solv.check()
print('a')
to_print=[check_timeout]

if checked_solv==sat:
    best_model = solv.model()
    best_distance = 1500#best_model.evaluate(max_dist).as_long()

    to_print.append(('solution 1) obj = ', best_distance, '; Time used: ', time.time()-starting_time))

    solv.add( Not( And([pred[i][j] for i in range(n) for j in range(n+m) if best_model.evaluate(pred[i][j])] + [v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])])) )
    solv.add( max_dist < best_distance)
    print('qui')
    check_timeout = timeout-time.time()
    solv.set('timeout', int(check_timeout*1000))
    checked_solv = solv.check()
    print('...')

while time.time() < timeout and checked_solv==sat:

    best_model = solv.model()
    best_distance = best_model.evaluate(max_dist).as_long()

    to_print.append(('Solution 1) Obj = ', best_distance, '; Time used: ', time.time()-starting_time))

    solv.add( max_dist < best_distance)
    solv.add( Not( And([pred[i][j] for i in range(n+m) for j in range(n+m) if best_model.evaluate(pred[i][j])] + [v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])])) )
    
    check_timeout = timeout-time.time() #time left
    solv.set('timeout', int(check_timeout*1000)) #time left in millisec
    
    checked_solv = solv.check()
    
ending_time = time.time()
print('----------------Recap----------------------')
print('Total time: ', to_print[0])
print('Time used: ', ending_time - starting_time)
print('')
for i in range(len(to_print)-1):
    for j in range(4):
        print(to_print[i+1][j], end='')
    print('')

print('') 
print('Best solution:')   
item_pred, cour_item = [(i,j) for j in range(n+m) for i in range(n+m) if best_model.evaluate(pred[i][j])], [(i, j) for j in range(n) for i in range(m) if best_model.evaluate(v[i][j])]
print_items_for_each_courier(cour_item, item_pred, sz, cpt)
print('Minimized max distance: ', best_distance)


start
qui
a
qui
...
----------------Recap----------------------
Total time:  300.0
Time used:  0.21301031112670898

solution 1) obj = 1500; Time used: 0.18900775909423828
Solution 1) Obj = 206; Time used: 0.21301031112670898

Best solution:
Corriere 1: 4 -> 0 -> 2 -> 4   Peso trasportato: 26 / 30
Corriere 0: 3 -> 1 -> 3   Peso trasportato: 17 / 18
Minimized max distance:  206


In [ ]:
time_for_main_search=0.5 #between 0 and 1
starting_time=time.time()
timeout = starting_time + 60*5*time_for_main_search
check_timeout = timeout-time.time()

print('Time for main search:', check_timeout)
solv, max_dist, pred, v = mcp(n,m,D,sz,cpt)

first_timstamp=time.time()
checked_solv = solv.check()
second_timestamp=time.time()

print('Time used for the first search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)

if checked_solv==sat:
    best_model = solv.model()
    best_distance = best_model.evaluate(max_dist).as_long()

    print('Time left:', timeout-time.time(), '; Distance: ', best_distance)
    print('--------------------------------------------------')

    solv.add( Not( And([v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])==True])) )
    solv.add( max_dist < best_distance)

    
    check_timeout = timeout-time.time()
    first_timstamp=time.time()
    checked_solv = solv.check()
    second_timestamp=time.time()
    print('Time used for search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)


while time.time() < timeout and checked_solv==sat:

    print('Current time left:', timeout-time.time(), '; Best distance found so far: ', best_distance)

    check_timeout = timeout-time.time()
    solv.set('timeout', int((timeout-time.time())*1000))

    best_model = solv.model()
    best_distance = best_model.evaluate(max_dist).as_long()
    solv.add( max_dist < best_distance)
    solv.add( Not( And( [v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])==True])) )

    first_timstamp=time.time()
    checked_solv = solv.check()
    second_timestamp=time.time()
    print('--------------------------------------------------')
    print('Time used for the current search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)

print('--------------------------------------------------')
print('End of first search. Time used: ', timeout-time.time(), '/', (starting_time + 60*5)*time_for_main_search)

#--------------------------------------------------------------------------------------------------------------------------------------------------------------

timeout = time.time() + 60*(1-time_for_main_search)*5
starting_time_2=time.time()
check_timeout = timeout - starting_time_2
print('Time for optimization search:', check_timeout)

solv_2, max_dist, pred, v = mcp(n,m,D,sz,cpt)
solv_2.add(And([v[i][j] for i in range(m) for j in range(m) if best_model.evaluate(v[i][j])]))
solv_2.add(max_dist <= best_distance)

first_timstamp=time.time()
checked_solv = solv_2.check()
second_timestamp=time.time()

print('Time used for the first search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)

if checked_solv==sat:
    best_model = solv_2.model()
    best_distance = best_model.evaluate(max_dist).as_long()

    print('Time left:', timeout-time.time(), '; Distance: ', best_distance)
    print('--------------------------------------------------')

    solv_2.add( Not(And([pred[i][j] for i in range(n+m) for j in range(n+m) if best_model.evaluate(pred[i][j])]) ))
    solv_2.add( max_dist < best_distance)

    check_timeout = timeout-time.time()
    solv_2.set('timeout', int(check_timeout*1000))
    first_timstamp=time.time()
    checked_solv = solv_2.check()
    second_timestamp=time.time()
    print('Time used for search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)


while time.time() < timeout and checked_solv==sat:

    print('Current time left:', timeout-time.time(), '; Best distance found so far: ', best_distance)
    check_timeout = timeout-time.time()
    solv_2.set('timeout', int(check_timeout*1000))
    best_model = solv_2.model()
    best_distance = best_model.evaluate(max_dist).as_long()

    solv_2.add( max_dist < best_distance)
    solv_2.add( Not( And( [pred[i][j] for i in range(n+m) for j in range(n+m) if best_model.evaluate(pred[i][j])])) )

    first_timstamp=time.time()
    checked_solv = solv_2.check()
    second_timestamp=time.time()
    print('--------------------------------------------------')
    print('Time used for the current search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)

print('--------------------------------------------------')
print('Best Solution Found:')
    
item_pred, cour_item = [(i,j) for j in range(n+m) for i in range(n+m) if best_model.evaluate(pred[i][j])], [(i, j) for j in range(n) for i in range(m) if best_model.evaluate(v[i][j])]
print_items_for_each_courier(cour_item, item_pred, sz, cpt)
print('Minimized max distance: ', best_distance)

print('Time left:', timeout-time.time())


Time for main search: 150.0
Time used for the first search: 10.055383682250977 / 150.0 ; Result:  sat
Time left: 135.6693935394287 ; Distance:  1356
--------------------------------------------------
Time used for search: 112.19898700714111 / 135.65439343452454 ; Result:  sat
Current time left: 23.455406427383423 ; Best distance found so far:  1356
--------------------------------------------------
Time used for the current search: 23.469698190689087 / 23.455406427383423 ; Result:  unknown
--------------------------------------------------
End of first search. Time used:  -0.08190774917602539 / 852770915.7018057
Time for optimization search: 150.0
Time used for the first search: 32.31840753555298 / 150.0 ; Result:  sat
Time left: 113.67331457138062 ; Distance:  1312
--------------------------------------------------
Time used for search: 31.868614435195923 / 113.6029167175293 ; Result:  sat
Current time left: 81.73128938674927 ; Best distance found so far:  1312
-----------------------

In [9]:
import random

def rnd_value(cour_list, times=1):
    tot=[]
    for rip in range(times):
        dists=[]
        for cour in range(m):
            random.shuffle(cour_list[cour])
            dists.append(D[n][cour_list[cour][0]] + sum([D[cour_list[cour][i]][cour_list[cour][i+1]] for i in range(len(cour_list[cour])-1)]) + D[cour_list[cour][-1]][n])
        tot.append(max(dists))
    return sum(tot)/times


#Primo Solver: ------------------------------------------------------------------------------------

cour_solv = Solver()
v=[[Bool(f"vehicle({i})_{j}")for j in range(n+m)]for i in range(m)]

#each pack have to be carried by exactly one courrier
for j in range(n):
    cour_solv.add(exactly_one_seq([v[i][j] for i in range(m)], f"v_{j}"))

for j in range(m):
    for i in range(m):
        if j==i:
           cour_solv.add(v[i][n+j])
        else:
            cour_solv.add(Not(v[i][n+j]))
   
   
#Constraint peso
for c in range(m):
    cour_solv.add(Sum([If(v[c][i], 1, 0)*sz[i] for i in range(n)]) <= cpt[c])


#Prima rierca: ---------------------------------------------------------------------------------------
    
time_for_main_search=0.5 #between 0 and 1
starting_time=time.time()
timeout = starting_time + 60*5*time_for_main_search
check_timeout = timeout-time.time()

print('a')

checked_solv = cour_solv.check()

if checked_solv==sat:
    best_model = cour_solv.model()
    v_mod = [(i,j) for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])]
    cour_list=[[] for i in range(m)]
    for item in v_mod:
        cour_list[item[0]].append(item[1])
    best_d = rnd_value(cour_list) 
    #cour_solv.add( Not( And([v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])==True])) )

    check_timeout = timeout-time.time()
    checked_solv = cour_solv.check()
i=0
while time.time() < timeout and checked_solv==sat:
    i+=1
    print(i, end= '-')
    current_model = cour_solv.model()
    v_mod = [(i,j) for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])]
    cour_list=[[] for i in range(m)]
    for item in v_mod:
        cour_list[item[0]].append(item[1])
    current_d = rnd_value(cour_list) 
    #cour_solv.add( Not( And([v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])==True])) )

    if current_d < best_d:
        best_model = current_model
        best_d = current_d
        print('')
        print(best_d)

    check_timeout = timeout-time.time()
    cour_solv.set('timeout', int((timeout-time.time())*1000))
    checked_solv = cour_solv.check()
    
      

a
1-2-
1535.0


In [ ]:
'''
# Given formula F, find the model the maximizes the value of X 
# using at-most M iterations.
def max(F, X, M):
    s = Solver()
    s.add(F)
    last_model  = None
    i = 0
    while True:
        r = s.check()
        if r == unsat:
            if last_model != None:
                return last_model
            else:
                return unsat
        if r == unknown:
            raise Z3Exception("failed")
        last_model = s.model()
        s.add(X > last_model[X])
        i = i + 1
        if (i > M):
            raise Z3Exception("maximum not found, maximum number of iterations was reached")

x, y = Ints('x y')
F = [x > 0, x < 10, x == 2*y]
print max(F, x, 10000)
'''

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (1234820171.py, line 25)

In [ ]:

'''
   for k in range(m):
      k_courier_path=[]

      found_one_item=False
      one_item=0
      while not(found_one_item) and one_item<m:
         print(k)
         if (k, one_item) in cour_item:
            found_one_item=True
            k_courier_path.append(one_item)
            print('aa', k_courier_path)
         else:
            one_item+=1
      
      #Find successor
      if found_one_item:
         current_item = one_item
         while (current_item!=n+k): #Finchè non arrivo al deposito
            found_one_item=False
            i=0
            while not(found_one_item): #Finchè non trovo il successore
               if (i, current_item) in item_pred:
                  found_one_item=True
                  current_item=i
                  k_courier_path.append(current_item)
                  print('aaa', k_courier_path)
               else:
                  i+=1
      
         #Find Predecessor
         current_item = one_item
         while (current_item!=n+k): #Finchè non arrivo al deposito
            found_one_item=False
            i=0   
            while not(found_one_item): #Finchè non trovo il successore
               if (current_item, i) in item_pred:
                  found_one_item=True
                  current_item=i
                  k_courier_path[:0]=[current_item]
                  print('aaa', k_courier_path)
               else:
                  i+=1
         

      print(k, '-esimo corriere: ', k_courier_path)
   '''

"\nfor k in range(m):\n   k_courier_path=[]\n\n   found_one_item=False\n   one_item=0\n   while not(found_one_item) and one_item<m:\n      print(k)\n      if (k, one_item) in cour_item:\n         found_one_item=True\n         k_courier_path.append(one_item)\n         print('aa', k_courier_path)\n      else:\n         one_item+=1\n   \n   #Find successor\n   if found_one_item:\n      current_item = one_item\n      while (current_item!=n+k): #Finchè non arrivo al deposito\n         found_one_item=False\n         i=0\n         while not(found_one_item): #Finchè non trovo il successore\n            if (i, current_item) in item_pred:\n               found_one_item=True\n               current_item=i\n               k_courier_path.append(current_item)\n               print('aaa', k_courier_path)\n            else:\n               i+=1\n   \n      #Find Predecessor\n      current_item = one_item\n      while (current_item!=n+k): #Finchè non arrivo al deposito\n         found_one_item=False\n 